In [1]:
###%pip package###whl###module###

try:
    import pandas as pd
    import time
except:
    %pip install pandas numpy matplotlib scipy tqdm requests_toolbelt pymysql sqlalchemy flask pandas_datareader  torch  pymssql EMD-signal websocket-client ccxt statsmodels  sklearn sshtunnel
else:
    print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+28800))}')    

2022-09-16 09:24:19


In [1]:
###日志 logger##20220825A######
import logging
import sys
import time

formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(name)s:%(filename)s:%(lineno)d:%(funcName)s:%(process)s: %(message)s')

def make_logger(name, log_level=logging.INFO, log_file="log.txt", file_mode="a"): #w写 a追加
    logger = logging.getLogger(name)
    logger.setLevel(level=log_level)
    handler = logging.FileHandler(log_file, mode=file_mode)
    handler.setLevel(level=log_level)
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(level=log_level)
    handler.setFormatter(formatter)
    # logger.addHandler(handler)
    return logger

name = 'wsHq'
curDate = time.strftime("%Y%m%d")
logger = make_logger(name, log_level=logging.DEBUG, log_file= "../logs/%s_%s.log" % (name, curDate) )

version = "1.0.1"
print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+28800))} ********** ')

Now: 1.0.1 : 2022-10-25 10:39:06 ********** 


In [3]:
###行情订阅 import####
import websocket
import time
import threading
import requests
import json
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime, timedelta
import hashlib
import hmac
import math
import sys
import logging
import os

# worthDir = '/home/at/py/ipynb/'
# dbfile = '%s/%s' % ('/mnt/e/milu/py', 'binance.db')

worthDir = '../pkl'
dbfile = '%s/%s' % (worthDir, 'binance.db')

curTime = int(time.time())
listenKeyTime = curTime  #<60
pongTime = curTime  #<10

curDateTime = time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+28800))

symbol = pair = 'BTCUSDT'
contractType = 'PERPETUAL'
interval = '1m'

df_btcusdt_1m = pd.DataFrame()


# serverUrl = 'http://testnet.binancefuture.com'
serverUrl = 'http://fapi.binance.com'  #www fapi
serverTimeadj = 10

url = f'{serverUrl}/fapi/v1/time'
timestamp = int(round((time.time() ) * 1000) )
res = requests.get(url)
rt = json.loads(res.text)

if 'code' in rt and 'msg' in rt:
    logger.error(f'res : {res.text}')
else:
    serverTimeadj = ((rt['serverTime']- timestamp) ) //1000


version = "1.0.1"
print(sys.version)
# print(sys.executable)
print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+28800))} +{serverTimeadj}*****行情订阅 import***** ')

3.8.10 (default, Jun 22 2022, 20:18:18) 
[GCC 9.4.0]
Now: 1.0.1 : 2022-09-16 09:24:27 +0*****行情订阅 import***** 


In [4]:
###行情订阅 ws##20220827A######
print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+28800))}')

logger.info(f'--websocket.WebSocketApp--{version}---') #debug  error info warning

def on_open(wsapp):
    logger.info(f'--Starting--')

def on_data(wsapp, frame_data, frame_opcode, frame_fin):
    # logger.info(f'{frame_data = } \n{frame_opcode = } \n{frame_fin = }')
    pass

def on_message(wsapp, data):

    global df_btcusdt_1m

    logger.info(f'{data = }')
    dk = json.loads(data)
    if dk['ps'] == symbol and dk['ct'] == contractType and dk['k']['i'] == interval and dk['k']['x'] == True:
        dk_df = pd.DataFrame(dk['k'], index=[0])

        dk_df.drop(columns = ['i', 'f', 'L', 'x', 'B', ], inplace=True)
        dk_df.columns = ['openTime','closeTime','open','close','high','low','vol','numTrade','Amt','bidVol','bidAmt'] #,'ignore'

        dk_df['open']  = pd.to_numeric(dk_df['open'] , errors='coerce', downcast='float')
        dk_df['high']  = pd.to_numeric(dk_df['high'] , errors='coerce', downcast='float')
        dk_df['low']  = pd.to_numeric(dk_df['low'] , errors='coerce', downcast='float')
        dk_df['close']  = pd.to_numeric(dk_df['close'] , errors='coerce', downcast='float')
        dk_df['vol']  = pd.to_numeric(dk_df['vol'] , errors='coerce', downcast='float')
        dk_df['Amt']  = pd.to_numeric(dk_df['Amt'] , errors='coerce', downcast='float')
        dk_df['bidVol']  = pd.to_numeric(dk_df['bidVol'] , errors='coerce', downcast='float')
        dk_df['bidAmt']  = pd.to_numeric(dk_df['bidAmt'] , errors='coerce', downcast='float')
        dk_df['openTime']  = pd.to_numeric(dk_df['openTime'] , errors='coerce', downcast='integer')
        dk_df['closeTime']  = pd.to_numeric(dk_df['closeTime'] , errors='coerce', downcast='integer')
        dk_df['numTrade']  = pd.to_numeric(dk_df['numTrade'] , errors='coerce', downcast='integer')

        dk_df['openDateTime'] = pd.to_datetime(dk_df['openTime'], unit="ms", origin="1970-01-01 08:00:00") 
        dk_df['closeDateTime'] = pd.to_datetime(dk_df['closeTime'], unit="ms", origin="1970-01-01 08:00:00") 
        # dk_df['time'] = pd.to_datetime(int(time.time()), unit='s', origin="1970-01-01 08:00:00")

        dk_df.sort_index(axis=1, ascending=True, inplace=True)

        print(f'{dk_df = }  \n')

        if  not df_btcusdt_1m.empty and dk['k']['t'] in df_btcusdt_1m['openTime'].values and dk['k']['T'] in df_btcusdt_1m['closeTime'].values:
            df_btcusdt_1m.sort_index(axis=1, ascending=True, inplace=True)
            df_btcusdt_1m.loc[ df_btcusdt_1m['closeTime'] == dk['k']['T'] ] = dk_df.to_numpy()
        else:
            df_btcusdt_1m = pd.concat([df_btcusdt_1m, dk_df ], ignore_index=True)

        df_btcusdt_1m.to_pickle('%s/%s.pkl' % (worthDir, "btcusdt_1m" ) )
        # df_btc = pd.read_pickle('%s/%s.pkl' % (worthDir, "btcusdt_1m" ) )

        conn = sqlite3.connect(dbfile)
        # df_btcusdt_1m = pd.read_sql(' select * from btcusdt_1m ', conn) 
        dk_df.to_sql('btcusdt_1m', con=conn, if_exists='append', index=False) #,
        conn.close()

def on_cont_message(wsapp, frame_data, frame_fin):
    # logger.info(f'{frame_data = } \n{frame_fin = }')
    pass
    
def on_ping(wsapp, frame_data):
    logger.info(f'{frame_data = }') 
    wsapp.send("", websocket.ABNF.OPCODE_PONG) # 接收到 PING 数据帧后, 需要立即给服务端回复 PONG 数据帧

def on_pong(wsapp, frame_data):
    # logger.warning(f'{frame_data = }') 
    pass
    
def on_error(wsapp, err):
    logger.error(f'{err = }') 
    pass

def on_close(wsapp, close_status_code, close_reason):
    logger.info(f'{close_status_code = } , {close_reason = }')
    pass

def getListenKey():
    listenKey_url = "http://testnet.binancefuture.com/fapi/v1/listenKey"
    api_key = "fcc2838327a124367acd634323b93b1fb53d6fc66e84d679169a78adcaf1bf3e"
    secret_key = "4f58b518cecdfe9c574cd2aa9cbbea429796673d6226424bdda1f0155cd78876"

    headers = {"x-mbx-apikey":api_key}
    res = requests.post(listenKey_url,  headers=headers) #, verify=False, proxies=proxy data={},
    rt = json.loads(res.text)
    # print(f'{rt["listenKey"]}') 
    return rt["listenKey"]

# ws_url = f'wss://stream.binancefuture.com/ws/{getListenKey()}'
# ws_url = f'wss://fstream.binance.com/ws/btcusdt_perpetual@continuousKline_1m'  #ok
# ws_url = f'wss://fstream.binance.com/ws/!markPrice@arr' 
# ws_url = f'wss://fstream.binancefuture.com/ws/btcusdt_perpetual@continuousKline_1m' #ok
# ws_url = f'wss://fstream.binancefuture.com/ws/bnbusdt@aggTrade'

url = f'wss://fstream.binancefuture.com'
streamName = f'{symbol.lower()}_{contractType.lower()}@continuousKline_{interval}'
ws_url = f'{url}/ws/{streamName}'

def wssApp():
    global wsapp
    wsapp = websocket.WebSocketApp(ws_url, on_open=on_open, on_data=on_data, on_message=on_message, on_cont_message=on_cont_message, on_ping=on_ping, on_pong=on_pong, on_error=on_error, on_close=on_close )

    wsapp.run_forever(ping_interval=30, ping_timeout=10)  # wsapp.close()

# t1 = threading.Thread(target=wssApp, args=(), kwargs={}, )
# # t1.setDaemon(True)  #False 同 t1.join()
# t1. start()


Now: 1.0.1 : 2022-09-16 09:53:03


In [ ]:
t1 = threading.Thread(target=wssApp, args=(), kwargs={}, )
t1. start()

In [21]:
# sys.exit('退出')
t1.is_alive()
wsapp.close()
t1.is_alive() #False
print(f'{t1.is_alive() = } , {t1.ident = } , {t1.name = } , ')

t1.is_alive() = True , t1.ident = 140131499443968 , t1.name = 'Thread-12' , 


In [ ]:
# d = {'t': 1662628560000, 'T': 1662628619999, 'i': '1m', 'f': 27185952694, 'L': 27185962986, 'o': '19205.30', 'c': '19205.30', 'h': '19205.70', 'l': '19205.30', 'v': '30.474', 'n': 13, 'x': True, 'q': '585262.50380', 'V': '0.479', 'Q': '9199.53030', 'B': '0'}
# print(f'{d = }')
# d_df = pd.DataFrame(d, index=[0])
# print(f'{d_df = }')

print(f'{df_btcusdt_1m = }')


In [5]:
###查询行情 kline###continuousKlines###
def continuousKlines(pair='btcusdt', contractType='PERPETUAL', interval='1m', startTime=0, endTime=0, limit=499 ):

    url = f'{serverUrl}/fapi/v1/continuousKlines'

    # pair = 'btcusdt'
    # contractType = 'PERPETUAL'
    # interval = '5m' #period #1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
    # startTime = 0
    # endTime = 0
    # limit = 3 #499

    if limit < 0 or startTime < 0 or endTime < 0 or (startTime > endTime):
        k_df = pd.DataFrame()
        logger.error(f'res : {{"code":-1130,"msg":"Data sent for parameter "limit" or "startTime" or "endTime" is not valid."}}')
        return k_df

    elif limit == 0:
        if startTime == 0 and endTime == 0:
            k_df = pd.DataFrame()
            logger.error(f'res : {{"code":-1130,"msg":"Data sent for parameter "limit" is not valid."}}') 
            return k_df

        elif startTime == 0:
            params = {"pair":pair, "contractType":contractType, "interval":interval, "endTime":endTime }          
        elif endTime == 0 :
            params = {"pair":pair, "contractType":contractType, "interval":interval,"startTime":startTime }  
    elif limit > 0:
        if startTime == 0 and endTime == 0:
            params = {"pair":pair, "contractType":contractType, "interval":interval, "limit":limit, }  
        elif startTime == 0:
            params = {"pair":pair, "contractType":contractType, "interval":interval, "endTime":endTime }          
        elif endTime == 0 :
            params = {"pair":pair, "contractType":contractType, "interval":interval,"startTime":startTime, "limit":limit, }   
        elif startTime > 0 and endTime > 0:  # startTime + limit < endTime
            params = {"pair":pair, "contractType":contractType, "interval":interval,"startTime":startTime, "endTime":endTime, "limit":limit, }   

    logger.info(f'{params=}')
    
    res = requests.get(url, params=params)

    rt = json.loads(res.text)

    if 'code' in rt and 'msg' in rt:
        k_df = pd.DataFrame()
        logger.error(f'res : {res.text}')
    else:
        k_df = pd.DataFrame(rt)
        if k_df.empty:
            return k_df    
        
        k_df.columns = ['openTime','open','high','low','close','vol','closeTime','Amt','numTrade','bidVol','bidAmt','ignore'] #
        
        k_df['open']  = pd.to_numeric(k_df['open'] , errors='coerce', downcast='float')
        k_df['high']  = pd.to_numeric(k_df['high'] , errors='coerce', downcast='float')
        k_df['low']  = pd.to_numeric(k_df['low'] , errors='coerce', downcast='float')
        k_df['close']  = pd.to_numeric(k_df['close'] , errors='coerce', downcast='float')
        k_df['vol']  = pd.to_numeric(k_df['vol'] , errors='coerce', downcast='float')
        k_df['Amt']  = pd.to_numeric(k_df['Amt'] , errors='coerce', downcast='float')
        k_df['bidVol']  = pd.to_numeric(k_df['bidVol'] , errors='coerce', downcast='float')
        k_df['bidAmt']  = pd.to_numeric(k_df['bidAmt'] , errors='coerce', downcast='float')
        k_df['openTime']  = pd.to_numeric(k_df['openTime'] , errors='coerce', downcast='integer')
        k_df['closeTime']  = pd.to_numeric(k_df['closeTime'] , errors='coerce', downcast='integer')
        k_df['numTrade']  = pd.to_numeric(k_df['numTrade'] , errors='coerce', downcast='integer')

        k_df['openDateTime'] = pd.to_datetime(k_df['openTime'], unit="ms") #, origin="1970-01-01 08:00:00" , unit="ms"
        k_df['closeDateTime'] = pd.to_datetime(k_df['closeTime']//1000, unit="s") 
        # k_df['time'] = pd.to_datetime(int(time.time()), unit='s', unorigin="1970-01-01 08:00:00")
        
        k_df.drop(['ignore'], axis=1, inplace=True)

        k_df.sort_index(axis=1, ascending=True, inplace=True)


    return k_df

#权重 [1,100)  1 ; [100, 500)	2 ; [500, 1000]	5 ;  > 1000	10 ;

# [
# 1661744100000,      // 开盘时间 openTime
# "18879.99",         // 开盘价 open
# "18900.00",         // 最高价 high
# "18878.98",         // 最低价 low
# "18896.13",         // 收盘价(当前K线未结束的即为最新价) close
# "492.363",          // 成交量 vol
# 1661744399999,      // 收盘时间 closeTime
# "9302145.66080",    // 成交额 Amt
# 1874,               // 成交笔数 numTrade
# "385.983",          // 主动买入成交量 bidVol
# "7292402.33267",    // 主动买入成交额 bidAmt
# "0"                 // 请忽略该参数 ignore
# ]

# k_df = continuousKlines(pair='btcusdt', contractType='PERPETUAL', interval='1m',limit=2, startTime =1661824259000, endTime =1661824559999 ) # 2
# k_df = continuousKlines(limit=2, endTime =1661824559999 ) # 1661824559999 ,2 
# k_df = continuousKlines(limit=2, startTime =1661824259000, endTime =1661824559999 ) # 2
# k_df = continuousKlines(startTime =1661824259000, endTime =1661824559999 ) # 5
# k_df = continuousKlines(limit=2, startTime =1661824259000, ) # 2
# k_df = continuousKlines(limit=0, startTime =0, endTime =0 ) #err 0 
# k_df = continuousKlines(limit=-2 ) # err 0
# k_df = continuousKlines(limit=2 ) # 2
# print(f'{k_df}')  #11s

print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+28800))} *****查询行情 kline###continuousKlines***** ')

Now: 1.0.1 : 2022-09-16 09:54:02 *****查询行情 kline###continuousKlines***** 


In [ ]:
k_df = continuousKlines(limit=2 ) # 2
print(f'{k_df.dtypes}')  #11s
print(f'{k_df}')  #11s

In [19]:
###查询历史行情 日kline###getDayUKline###
def getDayUKline(symbol="BTCUSDT", interval="1m", date="0", histDays=1, limit=499):
    
    if date == "0":
        startSec = int(datetime.strptime(datetime.now().strftime("%Y%m%d"), "%Y%m%d").strftime("%s"))
    else:
        startSec = int(datetime.strptime(date, "%Y%m%d").strftime("%s"))
    
    endSec = int( ( datetime.fromtimestamp(startSec)+ timedelta(days=histDays) ).strftime("%s") )

    intervalCoef = {'m':60, 'h':60*60, 'd': 60*60*24, 'w':60*60*24*7, 'M':60*60*24*30}
    intervalSec = int(interval[0:-1]) * intervalCoef[interval[-1] ]

    totalCount = math.ceil( (endSec - startSec )/intervalSec )
    logger.info(f'{interval=}, {startSec=}, {endSec=}, {intervalSec=}, {limit=}, {intervalSec*limit=}, {totalCount=}, {histDays=}')

    # ukdfHist = pd.DataFrame(columns=['tradeDate', 'openTime', 'closeTime', 'closeSec', 'open', 'close', ])
    ukdfHist = pd.DataFrame()

    count = 1
    for Sec in range(startSec, endSec, intervalSec*limit):

        stopSec = min(Sec+intervalSec*limit-1 , endSec-1)

        print(f'{count=}:', "continuousKlines(", f'{symbol=}', ',', f'{interval=}', ',', f'{limit=}', ',', f'{Sec*1000=}', ', endTime=', f'{stopSec*1000+999}' , ' )')

        ukRet = continuousKlines(pair=symbol, interval=interval, limit=limit, startTime=Sec*1000, endTime=stopSec*1000+999 )

        # kdf = pd.DataFrame([{'tradeDate': time.strftime("%Y%m%d", time.localtime(rs.closeTime/1000) ), 'openTime': time.strftime("%H%M%S", time.localtime(rs.openTime/1000) ), 'closeTime': time.strftime("%H%M%S", time.localtime(rs.closeTime/1000) ), 'closeSec': rs.closeTime//1000, 'open': rs.open, 'close': rs.close} for _, rs in ukRet.iterrows() ])

        # ukdfHist.append(kdf)
        ukdfHist = pd.concat([ukdfHist,ukRet],ignore_index=True)
        # del kdf

        count += 1
        # time.sleep(1)

    logger.info(f'ukdfHist.iloc[:5,:] :\n{ukdfHist.iloc[:5,:]}')
    logger.info(f'ukdfHist.iloc[-5:,:] :\n{ukdfHist.iloc[-5:,:]}')
  
    return ukdfHist

print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+28800))} *****查询历史行情 日kline###getDayUKline***** ')

Now: 1.0.1 : 2022-09-16 00:44:43 *****查询历史行情 日kline###getDayUKline***** 


In [20]:
df = getDayUKline( date="20220913")

print(f'df.iloc[:5,:] :\n{df.iloc[:5,:]}')
print(f'df.iloc[-5:,:] :\n{df.iloc[-5:,:]}')

print(df.shape)

conn = sqlite3.connect(dbfile)
# df_btcusdt_1m = pd.read_sql(' select * from btcusdt_1m ', conn) 
df.to_sql('btcusdt_1m', con=conn, if_exists='append', index=False) #,
conn.close()
 

count=1: continuousKlines( symbol='BTCUSDT' , interval='1m' , limit=499 , Sec*1000=1663027200000 , endTime= 1663057139999  )
count=2: continuousKlines( symbol='BTCUSDT' , interval='1m' , limit=499 , Sec*1000=1663057140000 , endTime= 1663087079999  )
count=3: continuousKlines( symbol='BTCUSDT' , interval='1m' , limit=499 , Sec*1000=1663087080000 , endTime= 1663113599999  )
df.iloc[:5,:] :
           Amt       bidAmt      bidVol         close       closeDateTime  \
0   9942716.00  5419100.000  242.044006  22398.000000 2022-09-13 00:00:59   
1   7099691.50  2584510.500  115.405998  22370.300781 2022-09-13 00:01:59   
2  14288044.00  5434780.500  243.177002  22362.900391 2022-09-13 00:02:59   
3   8354714.00  5237442.000  234.268005  22358.400391 2022-09-13 00:03:59   
4   3981997.75  1991083.625   89.060997  22360.300781 2022-09-13 00:04:59   

       closeTime          high           low  numTrade          open  \
0  1663027259999  22399.000000  22376.699219      3305  22386.000000   
1 

In [ ]:
conn = sqlite3.connect(dbfile)
# df_btcusdt_1m = pd.read_sql(' select * from btcusdt_1m ', conn) 
df_btcusdt_1m.to_sql('btcusdt_1m', con=conn, if_exists='append', index=False) #,
conn.close()

df_btcusdt_1m.to_pickle('%s/%s.pkl' % (worthDir, "btcusdtdf" ) )
# df_btcusdt_1m = pd.read_pickle('%s/%s.pkl' % (worthDir, "btcusdtdf" ) )

In [ ]:
###查询历史行情 kline###histUKline###
def histUKline(symbol="BTCUSDT", interval="1m", histDays=1, limit=998):

    intervalCoef = {'m':60, 'h':60*60, 'd': 60*60*24, 'w':60*60*24*7, 'M':60*60*24*30}
    intervalSec = int(interval[0:-1]) * intervalCoef[interval[-1] ]
    curSec = int(time.time())
    startSec = int(time.mktime(time.strptime(time.strftime("%Y%m%d", time.localtime(time.time()-60*60*24*histDays ) ), '%Y%m%d')))

    interval_pre = interval
    if interval_pre == '10m':
        interval = '5m'
        intervalSec = intervalSec//2

    totalCount = math.ceil( (curSec - startSec )/intervalSec )
    logger.info(f'{interval=}, {startSec=}, {curSec=}, {intervalSec=}, {limit=}, {intervalSec*limit=}, {totalCount=}, {histDays=}')

    # ukdfHist = pd.DataFrame(columns=['tradeDate', 'openTime', 'closeTime', 'closeSec', 'open', 'close', ])
    ukdfHist = pd.DataFrame()

    count = 1
    for Sec in range(startSec, curSec, intervalSec*limit):

        print(f'{count=}:', "continuousKlines(", f'{symbol=}', ',', f'{interval=}', ',', f'{limit=}', ',', f'{Sec*1000+999=}', ', endTime=', f'{(Sec+intervalSec*limit-1)*1000+999}' , ' )')

        ukRet = continuousKlines(pair=symbol, interval=interval, limit=limit, startTime=Sec*1000+999, endTime=(Sec+intervalSec*limit-1)*1000+999 )

        kdf = pd.DataFrame([{'tradeDate': time.strftime("%Y%m%d", time.localtime(rs.closeTime/1000) ), 'openTime': time.strftime("%H%M%S", time.localtime(rs.openTime/1000) ), 'closeTime': time.strftime("%H%M%S", time.localtime(rs.closeTime/1000) ), 'closeSec': rs.closeTime//1000, 'open': rs.open, 'close': rs.close} for _, rs in ukRet.iterrows() ])

        # ukdfHist.append(kdf)
        ukdfHist = pd.concat([ukdfHist,kdf],ignore_index=True)
        del kdf

        count += 1
        time.sleep(3)

    logger.info(f'ukdfHist.iloc[:5,:] :\n{ukdfHist.iloc[:5,:]}')
    logger.info(f'ukdfHist.iloc[-5:,:] :\n{ukdfHist.iloc[-5:,:]}')

    if interval_pre == '10m':
        interval = interval_pre
        intervalSec = int(intervalSec*2)
        rePeriod = '10T'

        ukdfHistCp = ukdfHist.copy()
        ukdfHistCp['dateTime'] = ukdfHistCp['closeSec'].apply(lambda x: datetime.fromtimestamp(x))
        ukdfHistCp = ukdfHistCp.set_index(keys=['dateTime'], drop=True)
        # ukdfHistCp = ukdfHistCp.reindex(ukdfHistCp['dateTime'].sort_values(ascending=True).index)

        logger.info(f'ukdfHistCp.iloc[:5,:] :\n{ukdfHistCp.iloc[:5,:]}')

        if not ukdfHistCp.empty:
            openSr = ukdfHistCp['open'].resample(rePeriod, label='right').first()  
            openTimeSr = ukdfHistCp['openTime'].resample(rePeriod, label='right').first()  
            closeSr = ukdfHistCp['close'].resample(rePeriod, label='right').last()  #last first max min
            closeTimeSr = ukdfHistCp['closeTime'].resample(rePeriod, label='right').last()  
            closeSecSr = ukdfHistCp['closeSec'].resample(rePeriod, label='right').last()  
            tradeDateSr = ukdfHistCp['tradeDate'].resample(rePeriod, label='right').last()  

            ukdf = pd.concat([tradeDateSr, openTimeSr, closeTimeSr, closeSecSr,openSr, closeSr], axis=1) 
            
        ukdf.columns = ['tradeDate', 'openTime', 'closeTime', 'closeSec', 'open', 'close']
        ukdf.reset_index(drop=True,inplace=True)

        if ukdf.iloc[-1]['closeSec'] - ukdf.iloc[-2]['closeSec'] < intervalSec:
            ukdf.drop([len(ukdf)-1],inplace=True)

        # ukdf = ukdfHistCp.copy()
        # logger.info(f'{ukdf.iloc[:5,:]=}')
        
        del ukdfHistCp

    else:
        ukdf = ukdfHist.copy()

    del ukdfHist

    ukdf['close'] = ukdf['close'].astype(float)
    ukdf['pct'] = ukdf['close']/ukdf['close'].shift(1)-1  #涨跌幅
    ukdf.fillna(0, inplace=True)

    logger.info(f'ukdf.iloc[:5,:] :\n{ukdf.iloc[:5,:]}')
    logger.info(f'ukdf.iloc[-5:,:] :\n{ukdf.iloc[-5:,:]}')

    return ukdf

print( f'Now: {version} : {time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+28800))} *****查询历史行情 kline###histUKline***** ')

In [ ]:
symbol = "BTCUSDT"
interval = "10m" #period #1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
histDays = 60  # 30 10 84 120
limit = 998 # 1500  499 998

uk_df = histUKline( interval="10m", histDays=7)
print(uk_df.shape)
print(uk_df)

# t1 = threading.Thread(target=wssApp, args=(), kwargs={}, )
# t1. start()
# stop_thread(thread)

In [10]:
# 停止线程 所有
for thread in threading.enumerate():
    print(f'{thread.getName() = }')
    
    if thread.getName() != 'MainThread':
        stop_thread(thread)

thread.getName() = 'MainThread'
thread.getName() = 'Heartbeat'
thread.getName() = 'Thread-2'


In [8]:
# 停止线程 使用ctypes强行杀掉线程 ctypes.pythonapi.PyThreadState_SetAsyncExc
import inspect
import ctypes
import threading

def terminate_thread(thread):
    if not thread.isAlive():
        return

    exc = ctypes.py_object(SystemExit)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(
        ctypes.c_long(thread.ident), exc)
    if res == 0:
        raise ValueError("nonexistent thread id")
    elif res > 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(thread.ident, None)
        raise SystemError("PyThreadState_SetAsyncExc failed")


def _async_raise(tid, exctype):

    tid = ctypes.c_long(tid)

    if not inspect.isclass(exctype):
        exctype = type(exctype)

    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(
        tid, ctypes.py_object(exctype))

    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        raise SystemError("PyThreadState_SetAsyncExc failed")


def stop_thread(thread):
    if not thread.is_alive():
        return
    _async_raise(thread.ident, SystemExit)
